# Run BigDFT and manipulate its output files

Job and Logfile classes

In [1]:
from mybigdft import Logfile

In [2]:
logname = "../../../tests/log.yaml"
log = Logfile.from_file(logname)

In [3]:
inp = log.inputparams

In [4]:
from mybigdft import Job

In [5]:
with Job(inputparams=inp, name="test") as job:
    job.run(nmpi=6, nomp=3, dry_run=True)
    job.clean(logfiles_dir=True)

/Users/maximemoriniere/Documents/Python/MyBigDFT/doc/source/notebooks
/Users/maximemoriniere/Documents/bigdft/bigdft/build-1997/install/bin/bigdft-tool --name test -n 6 ...


In [9]:
for attr in ["energy", "dipole", "posinp"]:
    print(f"{attr}\n\tjob: {getattr(job.logfile, attr)}\n\tlog: {getattr(log, attr)}")

energy
	job: None
	log: -19.884659235401838
dipole
	job: None
	log: [-0.00051199, -0.00051199, -0.00055711]
posinp
	job: 2   angstroem
free
N   2.97630782434901e-23   6.87220595204354e-23   0.0107161998748779
N  -1.10434491945017e-23  -4.87342174483075e-23   1.10427379608154

	log: 2   angstroem
free
N   2.97630782434901e-23   6.87220595204354e-23   0.0107161998748779
N  -1.10434491945017e-23  -4.87342174483075e-23   1.10427379608154



In [10]:
with Job(inputparams=inp, name="test") as job:
    job.run(nmpi=6, nomp=3, force_run=True)
    job.clean()

/Users/maximemoriniere/Documents/Python/MyBigDFT/doc/source/notebooks
mpirun -np 6 /Users/maximemoriniere/Documents/bigdft/bigdft/build-1997/install/bin/bigdft test ...
 <BigDFT> log of the run will be written in logfile: ./log-test.yaml



In [11]:
for attr in ["energy", "dipole", "posinp"]:
    print(f"{attr}\n\tjob: {getattr(job.logfile, attr)}\n\tlog: {getattr(log, attr)}")

energy
	job: -19.884659235399393
	log: -19.884659235401838
dipole
	job: [-0.00051199, -0.00051199, -0.00055711]
	log: [-0.00051199, -0.00051199, -0.00055711]
posinp
	job: 2   angstroem
free
N   2.97630782434901e-23   6.87220595204354e-23   0.0107161998748779
N  -1.10434491945017e-23  -4.87342174483075e-23   1.10427379608154

	log: 2   angstroem
free
N   2.97630782434901e-23   6.87220595204354e-23   0.0107161998748779
N  -1.10434491945017e-23  -4.87342174483075e-23   1.10427379608154



In [7]:
0/0

ZeroDivisionError: division by zero

In [ ]:
inp = InputParams({"posinp": {
    "units": "angstroem",
    "cell": [40, ".inf", ".inf"],
    "positions": [
        {'N': [2.97630782434901e-23, 6.87220595204354e-23, 0.0107161998748779]},
        {'N': [-1.10434491945017e-23, -4.87342174483075e-23, 1.10427379608154]},
    ]
}})
print(inp)
print(inp.posinp)
with Job(inputparams=inp, name="test") as job:
    job.run(nmpi=6, nomp=3, dry_run=True)
    job.clean()

In [ ]:
inp = InputParams({"posinp": {
    "units": "angstroem",
    "cell": [40, ".inf", ".inf"],
    "positions": [
        {'N': [0.0107161998748779, 2.97630782434901e-23, 6.87220595204354e-23]},
        {'N': [1.10427379608154, -1.10434491945017e-23, -4.87342174483075e-23]},
    ]
}})
print(inp)
print(inp.posinp)
with Job(inputparams=inp, name="test") as job:
    job.run(nmpi=6, nomp=3, force_run=True)
    job.clean()

In [ ]:
job.logfile.boundary_conditions